# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его.

In [1]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()

In [2]:
def lesk(word, sentence):
    
    bestsense = 0
    maxoverlap = 0
    
    # лемматизируем контекст (предложение)
    sentence_lemmatized = [lemmatizer.lemmatize(word) for word in sentence]
    
    # находим в wordnet синсеты для слова
    synsets = wn.synsets(word)
    
    # находим пересечения значений синсетов с контекстом 
    for i, synset in enumerate(synsets):
        definition = synset.definition().lower().split()
        definition_lemmatized = [lemmatizer.lemmatize(word) for word in definition]
        overlap = len(list(set(sentence_lemmatized) & set(definition_lemmatized)))
        if overlap > maxoverlap:
            maxoverlap, bestsense = overlap, i
       
    return bestsense

Чтобы проверить работу алгоритма, напишем функцию для извлечения слов и их контекстов.

In [3]:
def get_words_in_context(text, tokenizer=True, window=5):
    word2context = []
    if tokenizer:
        text_tokenized = [word.lower() for word in word_tokenize(text)]
    else:
        text_tokenized = text
    for i, word in enumerate(text_tokenized):
        word2context.append((word, text_tokenized[max(0, i-window):i] + text_tokenized[i+1:i+window]))
    return word2context

Проверим на каком-нибудь предложении:

In [4]:
text = 'Walk through the mountain gorge and rest on the river bank.'

In [5]:
for word2context in get_words_in_context(text):
    word, context = word2context
    sense = lesk(word, context)
    print('Word:', word)
    if wn.synsets(word):
        print('Sense in WordNet:', wn.synsets(word)[sense].definition())
    else:  
        print('Sense in WordNet: out of dictionary')
    print('\n')

Word: walk
Sense in WordNet: the act of traveling by foot


Word: through
Sense in WordNet: over the whole distance


Word: the
Sense in WordNet: out of dictionary


Word: mountain
Sense in WordNet: a land mass that projects well above its surroundings; higher than a hill


Word: gorge
Sense in WordNet: the passage between the pharynx and the stomach


Word: and
Sense in WordNet: out of dictionary


Word: rest
Sense in WordNet: euphemisms for death (based on an analogy between lying in a bed and in a tomb)


Word: on
Sense in WordNet: in operation or operational


Word: the
Sense in WordNet: out of dictionary


Word: river
Sense in WordNet: a large natural stream of water (larger than a creek)


Word: bank
Sense in WordNet: a financial institution that accepts deposits and channels the money into lending activities


Word: .
Sense in WordNet: out of dictionary




Алгоритм не справился с многозначными словами gorge, rest, bank.

**Проверьте насколько хорошо работает такой метод на датасете из семинара**

In [6]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

In [7]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу).

In [9]:
all_words = 0
sim_words = 0
for sent in corpus_wsd:
    try:
        sent_tokens = [word[2].lower() for word in sent]
        for i, word in enumerate(sent):
            if word[0]:
                all_words += 1
                context = sent_tokens[max(0, i-5):i] + sent_tokens[i+1:i+5]
                sense = lesk(word[1], context)
                # в датасете все многозначные слова присутствует в wordnet,
                # но на случай, если какого-то слова нет в словаре, добавим условие:
                if wn.synsets(word[1]):
                    if wn.synsets(word[1])[sense] == wn.lemma_from_key(word[0]).synset():
                        sim_words += 1
    # для исключения предложений из датасета, где у слов нет тэгов
    except IndexError:
        pass
print('Accuracy score:', sim_words/all_words)

Accuracy score: 0.37558198180173646


Возможно, на результат могло повлиять наличие стоп-слов. Попробуем не учитывать их при сопоставлении контекста слова и его определения. Для этого добавим условие в функцию lesk():

In [10]:
stopwords = stopwords.words('english')

def lesk_without_stopwords(word, sentence):
    
    bestsense = 0
    maxoverlap = 0
    
    # добавляем условие 'if word not in stopwords'
    sentence_lemmatized = [lemmatizer.lemmatize(word) for word in sentence if word not in stopwords]

    synsets = wn.synsets(word)

    for i, synset in enumerate(synsets):
        definition = synset.definition().lower().split()
        # добавляем условие 'if word not in stopwords'
        definition_lemmatized = [lemmatizer.lemmatize(word) for word in definition if word not in stopwords]
        overlap = len(list(set(sentence_lemmatized) & set(definition_lemmatized)))
        if overlap > maxoverlap:
            maxoverlap, bestsense = overlap, i
       
    return bestsense

In [11]:
all_words = 0
sim_words = 0
for sent in corpus_wsd:
    try:
        sent_tokens = [word[2].lower() for word in sent]
        for i, word in enumerate(sent):
            if word[0]:
                all_words += 1
                context = sent_tokens[max(0, i-5):i] + sent_tokens[i+1:i+5]
                num = lesk_without_stopwords(word[1], context)
                if wn.synsets(word[1])[num] == wn.lemma_from_key(word[0]).synset():
                    sim_words += 1
    except IndexError:
        pass
print('Accuracy score:', sim_words/all_words)

Accuracy score: 0.4687490882111432


Уже выглядит лучше. Посмотрим, как теперь разбирается наше предложение:

In [12]:
for word2context in get_words_in_context(text):
    word, context = word2context
    sense = lesk_without_stopwords(word, context)
    print('Word:', word)
    if wn.synsets(word):
        print('Sense in WordNet:', wn.synsets(word)[sense].definition())
    else:  
        print('Sense in WordNet: out of dictionary')
    print('\n')

Word: walk
Sense in WordNet: the act of traveling by foot


Word: through
Sense in WordNet: having finished or arrived at completion


Word: the
Sense in WordNet: out of dictionary


Word: mountain
Sense in WordNet: a land mass that projects well above its surroundings; higher than a hill


Word: gorge
Sense in WordNet: a deep ravine (usually with a river running through it)


Word: and
Sense in WordNet: out of dictionary


Word: rest
Sense in WordNet: something left after other parts have been taken away


Word: on
Sense in WordNet: in operation or operational


Word: the
Sense in WordNet: out of dictionary


Word: river
Sense in WordNet: a large natural stream of water (larger than a creek)


Word: bank
Sense in WordNet: sloping land (especially the slope beside a body of water)


Word: .
Sense in WordNet: out of dictionary




Теперь для слов gorge и bank значения определены верно. Для слова rest значение изменилось, но осталось неправильным. Попробуем посмотреть на все значения этого слова в WordNet:

In [13]:
for sense in wn.synsets('rest'):
    print(sense.definition())

something left after other parts have been taken away
freedom from activity (work or strain or responsibility)
a pause for relaxation
a state of inaction
euphemisms for death (based on an analogy between lying in a bed and in a tomb)
a support on which things can be put
a musical notation indicating a silence of a specified duration
not move; be in a resting position
take a short break from one's activities in order to relax
give a rest to
have a place in relation to something else
be at rest
stay the same; remain in a certain state
be inherent or innate in
put something in a resting position, as for support or steadying
sit, as on a branch
rest on or as if on a pillow
be inactive, refrain from acting


Заметно, что некоторые значения описываются 3-4 словами, что, конечно, недостаточно для совпадения с контекстом. Это можно отнести к очевидным минусам метода Леска (из вариантов улучшений метода -- использовать словари синонимов, учитывать н-граммы, чтобы находить однокоренные слова и т.п.).

### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро).

Для каждого эксперимента рассчитайте ARI.

Для эмбеддингов будем использовать [предобученную модель adagram](https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib').

In [14]:
import re
import pandas as pd
import numpy as np
import adagram
from pymorphy2 import MorphAnalyzer
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.metrics import adjusted_rand_score

morph = MorphAnalyzer()

Загружаем датасет, создаем функцию для препроцессинга контекстов.

In [16]:
df = pd.read_csv('train.csv', sep='\t')
df.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...


In [17]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [18]:
def preprocess(text):
    tokenized = re.findall('[А-Яа-яёЁA-Za-z0-9-]+', text.lower())
    lemmatized = [morph.parse(token)[0].normal_form for token in tokenized]
    return lemmatized

Посмотрим, для скольких значений слов приведены контексты (gold_sense_id). Эта информация понадобится для указания количества кластеров (например, в алгоритме KMeans).

In [19]:
for word in df.word.unique():
    print(word, df[df.word == word].gold_sense_id.unique())

замок [1 2]
лук [1 2]
суда [1 2]
бор [1 2]


Загружаем модель adagram и создадим функцию для получения усредненного эмбеддинга текста с использованием модели.

In [20]:
vm_adagram = adagram.VectorModel.load('model.joblib')

In [21]:
# у готовой модели размерность векторов 300, поэтому dim=300

def get_embedding_adagram(text, window, model=vm_adagram, dim=300):
    
    word2context = []
    
    for i in range(len(text)-1):
        word = text[i]
        context = text[max(0, i-window):i] + text[i+1:i+window]
        word2context.append((word, context))
        
    vectors = np.zeros((len(word2context), dim))
    
    for i, elem in enumerate(word2context):
        word, context = elem
        try:
            sense = model.disambiguate(word, context).argmax()
            vectors[i] = model.sense_vector(word, sense)
        except (KeyError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

Создадим функцию для кластеризации контекстов (с подсчетом метрики ARI).

In [22]:
def clusterize(clusterizer, corpus=grouped_df):
    
    ARI = []
    
    for key, _ in corpus:
        
        texts = corpus.get_group(key)['context'].apply(preprocess)
        X = np.zeros((len(texts), 300))
        
        for i, text in enumerate(texts):
            text = [word for word in text if word != key]
            X[i] = get_embedding_adagram(text=text, window=5)
            
        clusterizer.fit(X)
        labels = np.array(clusterizer.labels_)+1
        
        ARI.append(adjusted_rand_score(corpus.get_group(key)['gold_sense_id'], labels))
        
    return np.mean(ARI)

Поэкспериментируем с разными алгоритмами кластеризации.

In [23]:
clusterize(KMeans(n_clusters=2))

0.49803872540257665

KMeans даже с учетом явного указания количества кластеров показывает 0.49. Посмотрим, как работает другой алгоритм, где тоже можно задавать количество кластеров, -- AgglomerativeClustering.

In [24]:
clusterize(AgglomerativeClustering(n_clusters=2))

0.49814828906349823

Результат почти не отличается. Теперь взглянем на алгоритм AffinityPropogation. Попробуем подобрать оптимальный параметр preference:

In [25]:
for i in np.arange(-20, 0, 5):
    print('Preference:', i)
    print('ARI score:', clusterize(AffinityPropagation(damping=0.7, preference=i)))

Preference: -20
ARI score: 0.029354654102275248
Preference: -15
ARI score: 0.33150732871317123
Preference: -10
ARI score: 0.44789643381100785
Preference: -5
ARI score: 0.28799833320357526


При preference -10 ARI наилучший. Теперь попробуем подкрутить параметр damping.

In [26]:
for i in np.arange(0.5, 0.9, 0.1):
    print('Damping:', i)
    print('ARI score:', clusterize(AffinityPropagation(damping=i, preference=-10)))

Damping: 0.5
ARI score: 0.40338052911441835
Damping: 0.6
ARI score: 0.5395473972163697
Damping: 0.7
ARI score: 0.44789643381100785
Damping: 0.7999999999999999
ARI score: 0.4910811066157188


При параметрах preference=-10 и damping=0.6 AffinityPropagation работает лучше, чем остальные рассмотренные алгоритмы (ARI score равен 0.54).